### Serial version without using libraries

In [10]:
import random
import sys

def calc_pi(N):
    M = 0
    for i in range(N):
    # Simulate impact coordinates
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
    # True if impact happens inside the circle
        if x**2 + y**2 < 1.0:
            M += 1
    return 4 * M / N


num_trials = 10**6
#num_trials = int(sys.argv[1])

pi = calc_pi(num_trials)

print("\n \t Computing pi in serial: \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi))

%timeit -r3 calc_pi(num_trials)


 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.139588

492 ms ± 1.35 ms per loop (mean ± std. dev. of 3 runs, 1 loop each)


### Paralelización con multiprocessing

In [11]:
from multiprocessing import Pool

def calc_pi_partial(N):
    M = 0
    for _ in range(N):
        x = random.uniform(-1, 1)
        y = random.uniform(-1, 1)
        if x*x + y*y < 1.0:
            M += 1
    return M


In [12]:
def calc_pi_multiprocessing(N, nprocs):
    trials_per_proc = N // nprocs
    with Pool(processes=nprocs) as pool:
        results = pool.map(calc_pi_partial, [trials_per_proc]*nprocs)
    M = sum(results)
    return 4 * M / (trials_per_proc * nprocs)


In [13]:
ncores = 4

pi_mp = calc_pi_multiprocessing(num_trials, ncores)

print("\n \t Computing pi with multiprocessing : \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi_mp))

%timeit -r3 calc_pi_multiprocessing(num_trials, ncores)


 	 Computing pi with multiprocessing : 

	 For 1000000 trials, pi = 3.143768

125 ms ± 1.33 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)


### Paralelización con Numba

In [14]:
import numpy as np
from numba import njit, prange

In [15]:
@njit(parallel=True)
def calc_pi_numba_parallel(N):
    M = 0
    for i in prange(N):
        x = np.random.uniform(-1.0, 1.0)
        y = np.random.uniform(-1.0, 1.0)
        if x*x + y*y < 1.0:
            M += 1
    return 4.0 * M / N


In [16]:
calc_pi_numba_parallel(10)

pi_nb_par = calc_pi_numba_parallel(num_trials)
print("\n \t Computing pi with Numba parallel : \n")
print("\t For %d trials, pi = %f\n" % (num_trials,pi_nb_par))

%timeit -r3 calc_pi_numba_parallel(num_trials)


 	 Computing pi with Numba parallel : 

	 For 1000000 trials, pi = 3.143128

1.42 ms ± 13.7 µs per loop (mean ± std. dev. of 3 runs, 1,000 loops each)


### Resultados de la ejecución en hpc-bio-nikola-cpu

CPUs = 4
NUMBA_NUM_THREADS = 4

 	 Computing pi in serial: 

	 For 1000000 trials, pi = 3.143648

366 ms ± 345 μs per loop (mean ± std. dev. of 3 runs, 1 loop each)

 	 Computing pi with multiprocessing : 

	 For 1000000 trials, pi = 3.141500

88.7 ms ± 1.26 ms per loop (mean ± std. dev. of 3 runs, 10 loops each)

 	 Computing pi with Numba parallel : 

	 For 1000000 trials, pi = 3.140160

2.09 ms ± 1.11 μs per loop (mean ± std. dev. of 3 runs, 100 loops each)